<a href="https://colab.research.google.com/github/huzaifa003/Haystack/blob/main/HayStack_QA_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip 'data/lectures/Mufti Taqi Usmani and Sahih International data with haystack format' #just to unzip the zip

Streaming output truncated to the last 5000 lines.
  inflating: __MACOSX/Mufti Taqi Usmani and Sahih International data with haystack format/._QTMuftTaqUsmEng_46_16.txt  
  inflating: Mufti Taqi Usmani and Sahih International data with haystack format/QTMuftTaqUsmEng_46_17.txt  
  inflating: __MACOSX/Mufti Taqi Usmani and Sahih International data with haystack format/._QTMuftTaqUsmEng_46_17.txt  
  inflating: Mufti Taqi Usmani and Sahih International data with haystack format/QTMuftTaqUsmEng_46_18.txt  
  inflating: __MACOSX/Mufti Taqi Usmani and Sahih International data with haystack format/._QTMuftTaqUsmEng_46_18.txt  
  inflating: Mufti Taqi Usmani and Sahih International data with haystack format/QTMuftTaqUsmEng_46_19.txt  
  inflating: __MACOSX/Mufti Taqi Usmani and Sahih International data with haystack format/._QTMuftTaqUsmEng_46_19.txt  
  inflating: Mufti Taqi Usmani and Sahih International data with haystack format/QTMuftTaqUsmEng_46_2.txt  
  inflating: __MACOSX/Mufti Taqi U

In [ ]:
%%bash
#installation
pip install --upgrade pip
pip install farm-haystack[colab,inference]

In [32]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

In [33]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

doc_dir = "data/lectures/Mufti Taqi Usmani and Sahih International data with haystack format"
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

Updating BM25 representation...: 100%|██████████| 12479/12479 [00:00<00:00, 21538.34 docs/s]


{'documents': [<Document: {'content': "{'text': 'Did you not see how Allah has created seven heavens one upon another,', 'meta': {'translator': 'Mufti Taqi Usmani', 'surah_id': 71, 'ayah_no': 15, 'surah_name_en': 'Nooh (Nooh)', 'surah_name_ar': 'نُوح'}}", 'content_type': 'text', 'score': None, 'meta': {'_split_id': 0}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '7e6390995b2421a35f504f5a4c5f2b41'}>,
  <Document: {'content': "{'text': 'He said, “Then bring it, if you are truthful.”', 'meta': {'translator': 'Mufti Taqi Usmani', 'surah_id': 26, 'ayah_no': 31, 'surah_name_en': 'Ash-Shuara (The Poets)', 'surah_name_ar': 'الشعراء'}}", 'content_type': 'text', 'score': None, 'meta': {'_split_id': 0}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'd87c58612cc7dabb044f995bc0e6f7a7'}>,
  <Document: {'content': "{'text': 'and towards your Lord turn with eagerness.', 'meta': {'translator': 'Mufti Taqi Usmani', 'surah_id': 94, 'ayah_no': 8, 'surah_name_en': 'Al-Inshirah (Solace)', 

In [34]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [35]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [36]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [40]:
prediction = pipe.run(
    query="In the name of?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:13<00:00, 13.91s/ Batches]


In [41]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Allah', 'type': 'extractive', 'score': 0.918065071105957, 'context': "{'text': 'In the Name of Allah, the Most Beneficent, the Most Merciful.', 'meta': {'translator': 'Dr Mohsin Khan', 'surah_id': 1, 'ayah_no': 1, 'surah", 'offsets_in_document': [{'start': 25, 'end': 30}], 'offsets_in_context': [{'start': 25, 'end': 30}], 'document_ids': ['2038891be2fba0039502dc6fa74cbe69'], 'meta': {'_split_id': 0}}>,
             <Answer {'answer': 'Allah', 'type': 'extractive', 'score': 0.673072099685669, 'context': "{'text': 'With the name of Allah, the All-Merciful, the Very-Merciful.', 'meta': {'translator': 'Mufti Taqi Usmani', 'surah_id': 1, 'ayah_no': 1, 'sur", 'offsets_in_document': [{'start': 27, 'end': 32}], 'offsets_in_context': [{'start': 27, 'end': 32}], 'document_ids': ['1d8a5b376e5b75767db5ccc8e78167b8'], 'meta': {'_split_id': 0}}>,
             <Answer {'answer': 'your magnificent Lord', 'type': 'extractive', 'score': 0.6208571791648865, 'context': "{'

In [39]:
from haystack.utils import print_answers

print_answers(prediction, details="minimum")  ## Choose from `minimum`, `medium`, and `all`

'Query: In the name of?'
'Answers:'
[   {   'answer': 'Allah',
        'context': "{'text': 'In the Name of Allah, the Most Beneficent, the "
                   "Most Merciful.', 'meta': {'translator': 'Dr Mohsin Khan', "
                   "'surah_id': 1, 'ayah_no': 1, 'surah"},
    {   'answer': 'Allah',
        'context': "{'text': 'With the name of Allah, the All-Merciful, the "
                   "Very-Merciful.', 'meta': {'translator': 'Mufti Taqi "
                   "Usmani', 'surah_id': 1, 'ayah_no': 1, 'sur"},
    {   'answer': 'your magnificent Lord',
        'context': "{'text': 'So, pronounce the purity of the name of your "
                   "magnificent Lord.', 'meta': {'translator': 'Mufti Taqi "
                   "Usmani', 'surah_id': 69, 'ayah_no': 52, "},
    {   'answer': 'your most exalted Lord',
        'context': "{'text': 'Pronounce the purity of the name of your most "
                   "exalted Lord,', 'meta': {'translator': 'Mufti Taqi "
                  